In [1]:
from pyspark.sql import SparkSession 
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import pyspark.sql.functions
from pyspark.sql.functions import col,when
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [2]:
accident = pd.read_csv('/home/linuxu/Desktop/1.csv')
modifiedAccident = accident.dropna()

/home/linuxu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#filter
modifiedAccident1 = modifiedAccident[modifiedAccident['Road_Type'] >= 0]
modifiedAccident2 = modifiedAccident1[modifiedAccident1['Speed_limit'] >= 0]
modifiedAccident3 = modifiedAccident2[modifiedAccident2['Junction_Detail'] >= 0]
modifiedAccident4 = modifiedAccident3[modifiedAccident3['2nd_Road_Class'] >= 0]
modifiedAccident5 = modifiedAccident4[modifiedAccident4['Junction_Control'] >= 0]
modifiedAccident6 = modifiedAccident5[modifiedAccident5['Light_Conditions'] >= 0]
modifiedAccident7 = modifiedAccident6[modifiedAccident6['Weather_Conditions'] >= 0]
modifiedAccident8 = modifiedAccident7[modifiedAccident7['Road_Surface_Conditions'] >= 0]
modifiedAccident9 = modifiedAccident8[modifiedAccident8['Special_Conditions_at_Site'] >= 0]
modifiedAccident10 = modifiedAccident9[modifiedAccident9['1st_Road_Class'] >= 0]
#filter

In [4]:
#choose data ammount
subData = modifiedAccident10[0:1200000]

subData.to_csv('/home/linuxu/Desktop/subData.csv',index=False)

In [5]:
allColumns = ['Road_Type', 'Speed_limit', 'Junction_Detail', '2nd_Road_Class', 
              '1st_Road_Class', 'Junction_Control','Light_Conditions','Weather_Conditions'
              ,'Road_Surface_Conditions','Special_Conditions_at_Site']

In [8]:
avg = 0
sum = 0

i = 1
while i <= 10:
    
    #open a spark session
  spark = SparkSession.builder.appName('ml-bank').getOrCreate()

    #read the data
  df = spark.read.csv('/home/linuxu/Desktop/subData.csv', header = True, inferSchema = True)
    
    #random select from columns
  comb = random.sample(allColumns, k = 10)

    #print the combinations that was chosen
  print(comb)

    #open and write file
  fc = open("/home/linuxu/Desktop/CombinationsTried.txt","a+")
  sentence = str(i)+ ".  "+ ', '.join(map(str, comb))  + "\n"
  fc.write(sentence)
  fc.close()
    
      #binary
  df = df.withColumn("Accident_Severity", when(col("Accident_Severity")  == 3,0).otherwise(1))

    #select the first col to be accident sevirity and the rest are random
  df = df.select('Accident_Severity', comb[0], comb[1], comb[2], comb[3],comb[4],comb[5],comb[6],comb[7],comb[8],comb[9])
  cols = df.columns

    #diffrintiate between strings and int in the data
  categoricalColumns = []
  numericCols = []
  for cmb in range(10):
    if df.dtypes[cmb+1][1] == 'string':
      categoricalColumns.append(cols[cmb+1])
    elif df.dtypes[cmb+1][1] == 'int':
      numericCols.append(cols[cmb+1])


  stages = []
    
  for categoricalCol in categoricalColumns:
      stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
      encoder = OneHotEncoderEstimator(inputCols= [stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
      stages += [stringIndexer, encoder]

  label_stringIdx = StringIndexer(inputCol = 'Accident_Severity', outputCol = 'label')
  stages += [label_stringIdx]

  assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
  assembler = VectorAssembler(inputCols = assemblerInputs, outputCol="features")
  stages += [assembler]

  pipeline = Pipeline(stages = stages)
  pipelineModel = pipeline.fit(df)
    
  df = pipelineModel.transform(df)
  selectedCols = ['label', 'features'] + cols
    
  df = df.select(selectedCols)

    #split the DATA
  train, test = df.randomSplit([0.8, 0.2])
  print("Data splitted")

    
    ###################
####### algorithems ########
    ###################
    
  #Random Forest Classifier
  rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
  rfModel = rf.fit(train)
  predictionsRF = rfModel.transform(test)

  resultsRF = predictionsRF.select(['prediction', 'label'])
  predictionAndLabelsRF = resultsRF.rdd
  metricsRF = MulticlassMetrics(predictionAndLabelsRF)

  fr = open("/home/linuxu/Desktop/results.txt","a+")
  fr.write("Column Combination: "+ ', '.join(map(str, comb)) + "\n")
  fr.write("Results for Random Forest Algorithem: \n")
  fr.write("Accuracy: " + str((metricsRF.accuracy)*100) + "%\n")
  fr.write("Precision: " + str(metricsRF.weightedPrecision) + "\n")
  fr.write("Recall: " + str(metricsRF.weightedRecall) + "\n")
  fr.write("TruePositiveRate: " + str(metricsRF.weightedTruePositiveRate) + "\n")
  fr.write("FalsePositiveRate: " + str(metricsRF.weightedFalsePositiveRate) + "\n\n\n\n")
  fr.close()
    
  filename = "/home/linuxu/Desktop/comb" + str(i) + "_RF.csv"
  predictionsRF.toPandas().to_csv(filename)
  print("RF done")

  sum += (metricsRF.accuracy)*100
    
  spark.catalog.clearCache()
  del comb
  del categoricalColumns
  del numericCols
  del stages
  print(i, "pass completed")
  i = i + 1
    
#write max
fr = open("/home/linuxu/Desktop/results.txt","a+")
fr.write("Avarage is: " + str(sum/10))
fr.close()

['Light_Conditions', 'Weather_Conditions', '1st_Road_Class', 'Junction_Detail', 'Special_Conditions_at_Site', 'Speed_limit', 'Road_Type', 'Road_Surface_Conditions', '2nd_Road_Class', 'Junction_Control']
Data splitted
1 pass completed
['Speed_limit', 'Junction_Detail', 'Road_Surface_Conditions', 'Road_Type', 'Light_Conditions', 'Junction_Control', '2nd_Road_Class', 'Weather_Conditions', '1st_Road_Class', 'Special_Conditions_at_Site']
Data splitted
2 pass completed
['Road_Surface_Conditions', 'Junction_Control', 'Road_Type', 'Speed_limit', 'Special_Conditions_at_Site', 'Junction_Detail', '1st_Road_Class', 'Weather_Conditions', '2nd_Road_Class', 'Light_Conditions']
Data splitted
3 pass completed
['Junction_Detail', 'Road_Type', 'Weather_Conditions', '2nd_Road_Class', 'Speed_limit', 'Special_Conditions_at_Site', 'Junction_Control', '1st_Road_Class', 'Light_Conditions', 'Road_Surface_Conditions']
Data splitted
4 pass completed
['Speed_limit', 'Weather_Conditions', 'Junction_Control', '2nd_R